In [ ]:
#Import libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore')
import sklearn
from sklearn.ensemble import RandomForestClassifier

import sys
sys.path
sys.path.append('/Users/matthewmcteer/opt/anaconda3/lib/python3.7/site-packages')

import lightgbm
import shap
import xgboost as xgb

from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import time
import missingno as msno

import lightgbm as lgbm
import optuna


# Response 11

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_11'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_11'].notna()]

df['response_11'].value_counts()


In [ ]:
df.shape

In [ ]:
df = df.drop(['response_11'], axis=1)


In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [ ]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [105]:
response11_idx = mod_df.index.tolist()
response11_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8717,
 8722,
 8724,
 8725,
 8728,
 8729,
 8730,
 8733,
 8736,
 8737,
 8741,
 

# Response 12

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_7'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_7 = NaN
df = df[features]
df = df[df['response_7'].notna()]

df['response_7'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_7'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [ ]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [121]:
response12_idx = mod_df.index.tolist()
response12_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5541,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6315,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7001,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8715,
 8717,
 8722,
 8724,
 8725,
 8728,
 

# Response 13

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_13'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_13 = NaN
df = df[features]
df = df[df['response_13'].notna()]

df['response_13'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_13'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [124]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [126]:
response13_idx = mod_df.index.tolist()
response13_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8717,
 8722,
 8724,
 8725,
 8728,
 8729,
 8730,
 8733,
 8736,
 8737,
 8741,
 

# Response 14

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_14'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_14 = NaN
df = df[features]
df = df[df['response_14'].notna()]

df['response_14'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_14'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [130]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [132]:
response14_idx = mod_df.index.tolist()
response14_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4441,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8717,
 8722,
 8724,
 8725,
 8728,
 8729,
 8730,
 8733,
 

# Response 15 

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_15'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_15 = NaN
df = df[features]
df = df[df['response_15'].notna()]

df['response_15'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_15'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [ ]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [138]:
response15_idx = mod_df.index.tolist()
response15_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4441,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8717,
 8722,
 8724,
 8725,
 8728,
 8729,
 8730,
 8733,
 

# Response 16 

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_16'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_16 = NaN
df = df[features]
df = df[df['response_16'].notna()]

df['response_16'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_16'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [141]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [143]:
response16_idx = mod_df.index.tolist()
response16_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4441,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8717,
 8722,
 8724,
 8725,
 8728,
 8729,
 8730,
 8733,
 

# Response 17 

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_17'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_17 = NaN
df = df[features]
df = df[df['response_17'].notna()]

df['response_17'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_17'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [146]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [148]:
response17_idx = mod_df.index.tolist()
response17_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4441,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5263,
 5268,
 5269,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5541,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6315,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7001,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8699,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8715,
 

# Response 18 

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_18'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_18 = NaN
df = df[features]
df = df[df['response_18'].notna()]

df['response_18'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_18'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [ ]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [153]:
response18_idx = mod_df.index.tolist()
response18_idx

[156,
 218,
 224,
 230,
 231,
 235,
 251,
 256,
 258,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1054,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1162,
 1185,
 1219,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1385,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2098,
 2101,
 2105,
 4126,
 4202,
 4205,
 4441,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4544,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4556,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4571,
 4574,
 4575,
 4577,
 4579,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4626,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4656,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5227,
 5229,
 5230,
 5235,
 5243,
 5256,
 5259,
 5263,
 5268,
 5269,
 5273,
 5275,
 5281,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5504,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5530,
 5533,
 5535,
 5537,
 5539,
 5541,
 5545,
 5550,
 5554,
 5555,
 5556,
 5562,
 5563,
 5566,
 5571,
 5574,
 5580,
 5585,
 5586,
 5587,
 5589,
 5594,
 5595,
 5600,
 5602,
 5608,
 5610,
 5618,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5821,
 5827,
 5830,
 5836,
 5840,
 5845,
 5852,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5930,
 5934,
 5939,
 5944,
 5964,
 5970,
 5972,
 5981,
 5986,
 5997,
 5998,
 6006,
 6015,
 6016,
 6018,
 6022,
 6031,
 6033,
 6035,
 6037,
 6045,
 6047,
 6053,
 6054,
 6056,
 6060,
 6067,
 6072,
 6076,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6126,
 6134,
 6158,
 6162,
 6168,
 6179,
 6200,
 6204,
 6227,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6307,
 6315,
 6317,
 6320,
 6327,
 6328,
 6329,
 6331,
 6348,
 6353,
 6354,
 6365,
 6367,
 6368,
 6373,
 6374,
 6390,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6437,
 6440,
 6449,
 6463,
 6465,
 6469,
 6503,
 6507,
 6513,
 6521,
 6532,
 6537,
 6545,
 6550,
 6555,
 6562,
 6572,
 6587,
 6593,
 6629,
 6635,
 6664,
 6669,
 6674,
 6681,
 6693,
 6703,
 6713,
 6718,
 6723,
 6726,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6830,
 6835,
 6858,
 6862,
 6871,
 6885,
 6889,
 6893,
 6896,
 6909,
 6914,
 6921,
 6929,
 6942,
 6953,
 6959,
 6984,
 6987,
 7001,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7193,
 7197,
 7204,
 7208,
 7210,
 7211,
 7225,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8438,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8496,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8583,
 8584,
 8585,
 8588,
 8590,
 8592,
 8595,
 8597,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8616,
 8618,
 8619,
 8622,
 8625,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8696,
 8699,
 8700,
 8701,
 8706,
 8708,
 8709,
 8711,
 8713,
 8715,
 

# Response 20 

In [ ]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

features_list = {
    'std_clinical_features' :[#'CPH_EV_AGE_CALC',
           #'TBL.PATIENT.INFO..PI_BL_GENDER',
           #'CPH_EV_CI_BMI_CALC',
           #'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            #'insulin_resistance',
            #'hypertensive',
            #'waist_to_hip_ratio',
            #'idf_metabolic_syndrome',
            #'eGFR',
            #'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            #'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            #'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            #'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            #'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            #'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           #'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            #'TBL.ALL.EVENTS..AE_BR_IGA',
            #'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            #'FIB4',
            #'NFS',
            #'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            #'BARD',
            #'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_20'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_20 = NaN
df = df[features]
df = df[df['response_20'].notna()]

df['response_20'].value_counts()

df.shape


In [ ]:
df = df.drop(['response_20'], axis=1)

#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar


In [156]:
# Delete rows containing either 75% or more than 75% NaN Values
perc = 33.5 # Here N is 75
min_count =  int(((100-perc)/100)*df.shape[1] + 1)
mod_df = df.dropna( axis=0, 
                    thresh=min_count)

In [ ]:
mod_df.shape

In [158]:
response20_idx = mod_df.index.tolist()
response20_idx

[156,
 218,
 224,
 230,
 231,
 251,
 256,
 494,
 500,
 508,
 513,
 518,
 524,
 529,
 533,
 541,
 545,
 551,
 555,
 566,
 567,
 705,
 714,
 716,
 721,
 722,
 730,
 733,
 738,
 745,
 759,
 762,
 803,
 814,
 849,
 867,
 871,
 905,
 906,
 910,
 912,
 948,
 962,
 965,
 996,
 1032,
 1034,
 1038,
 1039,
 1042,
 1044,
 1047,
 1051,
 1055,
 1056,
 1059,
 1087,
 1106,
 1143,
 1144,
 1185,
 1273,
 1305,
 1308,
 1309,
 1322,
 1327,
 1371,
 1372,
 1374,
 1379,
 1380,
 1390,
 1391,
 1395,
 1396,
 1399,
 2088,
 2092,
 2095,
 2101,
 2105,
 4126,
 4202,
 4205,
 4445,
 4521,
 4522,
 4523,
 4526,
 4529,
 4531,
 4532,
 4534,
 4535,
 4537,
 4538,
 4539,
 4540,
 4542,
 4543,
 4546,
 4547,
 4550,
 4551,
 4552,
 4554,
 4555,
 4558,
 4559,
 4560,
 4561,
 4565,
 4568,
 4569,
 4575,
 4577,
 4581,
 4589,
 4593,
 4596,
 4601,
 4608,
 4615,
 4617,
 4618,
 4619,
 4621,
 4623,
 4627,
 4629,
 4638,
 4639,
 4641,
 4643,
 4646,
 4647,
 4653,
 4657,
 4659,
 4660,
 4661,
 4662,
 4664,
 4665,
 4683,
 4684,
 4688,
 4692,
 4693,
 4695,
 4696,
 5169,
 5174,
 5176,
 5178,
 5184,
 5195,
 5196,
 5211,
 5214,
 5215,
 5216,
 5219,
 5224,
 5230,
 5235,
 5256,
 5259,
 5273,
 5275,
 5287,
 5294,
 5297,
 5308,
 5330,
 5341,
 5345,
 5347,
 5354,
 5363,
 5364,
 5374,
 5375,
 5377,
 5391,
 5393,
 5414,
 5419,
 5449,
 5484,
 5485,
 5502,
 5507,
 5509,
 5510,
 5511,
 5513,
 5514,
 5516,
 5521,
 5522,
 5524,
 5533,
 5535,
 5545,
 5550,
 5554,
 5563,
 5566,
 5571,
 5580,
 5585,
 5586,
 5587,
 5589,
 5595,
 5600,
 5602,
 5608,
 5610,
 5622,
 5627,
 5629,
 5630,
 5632,
 5634,
 5635,
 5639,
 5640,
 5644,
 5645,
 5646,
 5647,
 5648,
 5785,
 5790,
 5814,
 5827,
 5830,
 5836,
 5840,
 5845,
 5866,
 5870,
 5876,
 5901,
 5909,
 5914,
 5920,
 5934,
 5944,
 5964,
 5970,
 5972,
 5981,
 5997,
 5998,
 6006,
 6015,
 6018,
 6031,
 6035,
 6037,
 6047,
 6053,
 6054,
 6060,
 6067,
 6072,
 6079,
 6087,
 6093,
 6099,
 6108,
 6119,
 6134,
 6162,
 6168,
 6179,
 6200,
 6236,
 6237,
 6250,
 6262,
 6297,
 6299,
 6320,
 6327,
 6328,
 6353,
 6354,
 6368,
 6373,
 6401,
 6403,
 6409,
 6414,
 6423,
 6424,
 6430,
 6436,
 6440,
 6449,
 6463,
 6503,
 6507,
 6513,
 6521,
 6532,
 6545,
 6550,
 6562,
 6572,
 6587,
 6593,
 6629,
 6664,
 6669,
 6674,
 6681,
 6703,
 6713,
 6718,
 6723,
 6736,
 6771,
 6777,
 6789,
 6792,
 6803,
 6835,
 6858,
 6871,
 6885,
 6893,
 6896,
 6909,
 6929,
 6953,
 6959,
 6984,
 6987,
 7010,
 7030,
 7033,
 7064,
 7067,
 7072,
 7078,
 7080,
 7088,
 7096,
 7099,
 7107,
 7109,
 7112,
 7126,
 7160,
 7204,
 7208,
 7211,
 7226,
 7229,
 7237,
 7250,
 7251,
 8285,
 8287,
 8290,
 8293,
 8301,
 8308,
 8314,
 8316,
 8323,
 8329,
 8339,
 8349,
 8350,
 8352,
 8353,
 8354,
 8355,
 8360,
 8362,
 8363,
 8364,
 8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8373,
 8376,
 8379,
 8380,
 8381,
 8382,
 8383,
 8386,
 8391,
 8392,
 8394,
 8398,
 8404,
 8411,
 8412,
 8413,
 8418,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8430,
 8431,
 8432,
 8435,
 8436,
 8442,
 8452,
 8457,
 8463,
 8464,
 8475,
 8478,
 8497,
 8501,
 8506,
 8510,
 8513,
 8518,
 8520,
 8525,
 8539,
 8541,
 8544,
 8547,
 8550,
 8553,
 8556,
 8560,
 8561,
 8563,
 8566,
 8568,
 8572,
 8575,
 8577,
 8580,
 8584,
 8585,
 8588,
 8592,
 8595,
 8599,
 8602,
 8606,
 8607,
 8609,
 8611,
 8613,
 8614,
 8618,
 8619,
 8622,
 8627,
 8630,
 8632,
 8637,
 8639,
 8641,
 8643,
 8644,
 8658,
 8678,
 8680,
 8685,
 8693,
 8700,
 8708,
 8713,
 8724,
 8725,
 8728,
 8729,
 8730,
 8737,
 8741,
 8742,
 8747,
 9084,
 9085,
 9086,
 9088,
 9090,
 9091,
 9102,
 9103,
 9104,
 9107,
 9110,
 9112,
 9113,
 9114,
 9115,
 9116,
 9117,
 9118,
 9119,
 9121,
 9123,
 9125,
 9126,
 9128,
 9129,
 9130,
 9131,
 9134,
 9135,
 9139,
 9140,
 9142,
 9148,
 9150,
 9151,
 9152,
 9153,
 9156]